# Rossmann Dataset Preparation

This notebook creates the `rossmann_train_rich_top_100.csv` example dataset
that is distributed with `hcrystalball`.

Input files:

- https://www.kaggle.com/c/rossmann-store-sales/data?select=train.csv (use download all button and unzip it)
- https://storage.googleapis.com/kaggle-forum-message-attachments/96497/3097/store_states.csv

## Data fields

See https://www.kaggle.com/c/rossmann-store-sales/data

In [ ]:
from pathlib import Path
import pandas as pd
import hcrystalball.data as hcb_data

## Storing data

In [ ]:
# update data_path to where rossmann-store-sales folder is. Make sure store_states.csv is there as well
data_path = Path('rossmann-store-sales')
train_path = Path(data_path, 'train.csv')
store_path = Path(data_path, 'store.csv')
store_states_path = Path(data_path, 'store_states.csv')

In [ ]:
df_train = pd.read_csv(train_path, parse_dates=['Date'], dtype={'Store':int, 
                                                                'DayOfWeek':int, 
                                                                'Sales':int, 
                                                                'Customers':int, 
                                                                'Open':bool, 
                                                                'Promo':bool, 
                                                                'StateHoliday':str, 
                                                                'SchoolHoliday':bool})
df_store = pd.read_csv(store_path, dtype={'Store':int, 
                                          'StoreType':str, 
                                          'Assortment':str, 
                                          'CompetitionDistance':float, 
                                          'CompetitionOpenSinceMonth':float,
                                          'CompetitionOpenSinceYear':float,
                                          'Promo2':bool,
                                          'Promo2SinceWeek':float,
                                          'Promo2SinceYear':float,
                                          'PromoInterval':str
                                          })
df_store_states = pd.read_csv(store_states_path, dtype={'Store':int, 
                                                        'State':str})

In [ ]:
df_train_rich = (df_train[['Date','Store', 'Sales', 'Open', 'Promo', 'SchoolHoliday']]
                         .merge(df_store[['Store', 'StoreType', 'Assortment','Promo2']], on='Store', how='left')
                         .merge(df_store_states, on='Store', how='left')
                         .replace({'State':{'HB,NI':'HB'}})
                         .assign(HolidayCode=lambda x:'DE-'+x['State'])
                         .set_index('Date')
                )

In [ ]:
df_train_rich_top_100 = df_train_rich[df_train_rich['Store'].isin(df_train_rich.groupby('Store')['Sales'].sum().nlargest(100).index)]

In [ ]:
df_train_rich_top_100.to_csv(Path(hcb_data.__path__._path[0], 'rossmann_train_rich_top_100.csv'))

## Example data read

In [ ]:
from hcrystalball.utils import get_sales_data

In [ ]:
df = get_sales_data(n_dates=200, n_assortments=2, n_states=2, n_stores=2)
df.head()